In [19]:
%%bash
set -euo pipefail

# 0) Activate your conda env inside this bash cell
source /opt/anaconda3/etc/profile.d/conda.sh || true
conda activate OntologyAgent || true

# 1) Paths
LMSS="$HOME/legal-agent/imports/LMSS.owl"
[ -f "$LMSS" ] || { echo "LMSS not found at: $LMSS"; exit 1; }

# 2) Ensure CLI shims are on PATH (without assuming CONDA_PREFIX exists)
if [ -n "${CONDA_PREFIX:-}" ] && [ -d "$CONDA_PREFIX/bin" ]; then
  export PATH="$CONDA_PREFIX/bin:$PATH"
fi
export PATH="$HOME/.local/bin:$PATH"

# 3) Make sure runoak is available
python -m pip show oaklib >/dev/null 2>&1 || python -m pip install -U oaklib rdflib >/dev/null
command -v runoak || { echo "runoak not found on PATH"; exit 1; }

# 4) Try reading OWL directly; if it fails, convert to TTL with ROBOT and retry
if ! runoak -i "$LMSS" info >/dev/null 2>&1; then
  echo "Direct OWL read failed → converting to TTL with ROBOT…"
  mkdir -p "$HOME/legal-agent/tools"
  [ -f "$HOME/legal-agent/tools/robot.jar" ] || curl -L -o "$HOME/legal-agent/tools/robot.jar" \
    "https://github.com/ontodev/robot/releases/latest/download/robot.jar"
  command -v java >/dev/null 2>&1 || { echo "Java not found. On mac: brew install openjdk"; exit 1; }
  java -jar "$HOME/legal-agent/tools/robot.jar" convert \
    --input "$LMSS" --format ttl --output "$HOME/legal-agent/imports/LMSS.ttl"
  runoak -i "$HOME/legal-agent/imports/LMSS.ttl" info | head -n 20
else
  runoak -i "$LMSS" info | head -n 20
fi


/opt/anaconda3/envs/OntologyAgent/bin/runoak
